In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Pinecone
from langchain.schema import AIMessage, HumanMessage

In [2]:

# Load credentials from environment variables
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

if not all([COHERE_API_KEY, PINECONE_API_KEY, GROQ_API_KEY]):
    raise ValueError("Missing API keys. Please check your environment variables.")

In [4]:

# Initialize embeddings and index
embeddings = CohereEmbeddings()
index_name = "index-leapfrog-confluence"

try:
    docsearch = Pinecone.from_existing_index(index_name, embeddings)
    retriever = docsearch.as_retriever()
except Exception as e:
    raise ValueError(f"Error retrieving index '{index_name}': {str(e)}")

In [5]:

# Initialize the language model
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

In [6]:

# Get the question from the user
question = input("What is leapfrog education policy?")

# Retrieve relevant documents based on the question
context = retriever.get_relevant_documents(question)
context_str = "\n".join([doc.page_content for doc in context])

In [8]:
context

[Document(page_content='0\n\n2\n\n3'),
 Document(page_content='🗒️ Self Reflection\n\nDate: 2023-09-05T04:36:25.362Z\n\nLeapfrog - Home\nKnowledge Center\nPulse\nPulse Flow'),
 Document(page_content='Leapfrog Code of Conduct\n\nDate: 2024-02-25T11:52:56.470Z\n\nLeapfrog - Home\nPolicies, Standards and Guidelines'),
 Document(page_content='New Employee Agreement Signing\n\nDate: 2023-12-08T10:35:53.867Z\n\nLeapfrog - Home\nPolicies, Standards and Guidelines')]

In [9]:

# Define the system prompt
system = f'''
<rules>
Answer the QUESTION related to "Leapfrog Technology" that is in the CONTEXT.
Answer casual greetings and conversation questions.
    For example,
        Human: Hey!
        AI: Hello! How can I help?
</rules>
You are AskRibby, an AI chatbot created by `Leapfrog Technology` which provides answers following the <rules></rules>.
CONTEXT for my QUESTION is provided below.

{context_str}
'''

In [15]:
question='What is leapfrog education policy?'
question

'What is leapfrog education policy?'

In [16]:
# Prepare the human prompt
human = f"Human: {question}"

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | llm

chain


# try:
#     # Stream the response
#     partial_message = ""
#     for chunk in chain.stream({"text": question}):
#         if chunk.content is not None:
#             partial_message += chunk.content
#             print(partial_message, end="\r")
#     print()
# except Exception as e:
#     print(f"Error processing the question: {str(e)}")

ChatPromptTemplate(input_variables=[], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n<rules>\nAnswer the QUESTION related to "Leapfrog Technology" that is in the CONTEXT.\nAnswer casual greetings and conversation questions.\n    For example,\n        Human: Hey!\n        AI: Hello! How can I help?\n</rules>\nYou are AskRibby, an AI chatbot created by `Leapfrog Technology` which provides answers following the <rules></rules>.\nCONTEXT for my QUESTION is provided below.\n\n0\n\n2\n\n3\n🗒️ Self Reflection\n\nDate: 2023-09-05T04:36:25.362Z\n\nLeapfrog - Home\nKnowledge Center\nPulse\nPulse Flow\nLeapfrog Code of Conduct\n\nDate: 2024-02-25T11:52:56.470Z\n\nLeapfrog - Home\nPolicies, Standards and Guidelines\nNew Employee Agreement Signing\n\nDate: 2023-12-08T10:35:53.867Z\n\nLeapfrog - Home\nPolicies, Standards and Guidelines\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Human: What is leapfrog education p